In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=50)

# 텍스트 파일을 load -> List[Document] 형태로 변환
loader1 = TextLoader("data/nlp-keywords.txt")
loader2 = TextLoader("data/finance-keywords.txt")

# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

# 문서 개수 확인
len(split_doc1), len(split_doc2)

In [ ]:
# DB 생성
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")           # 임베딩 모델 선택 가능
db = Chroma.from_documents(
    documents=split_doc1, embedding=embeddings, persist_directory='./chroma_db', collection_name="my_db"
)

# persist_directory : 디스크 저장 시 저장경로
# collection_name : 폴더명

In [ ]:
# DB 로드 할 때는 아래와 같이
persist_db = Chroma(
    persist_directory='./chroma_db',
    embedding_function=OpenAIEmbeddings(),
    collection_name="my_db"
)

persist_db.get()            # 확인용

In [ ]:
# DB 컬렉션에 저장하기
db = Chroma.from_texts(
    ["안녕하세요. 반갑습니다.", "제 이름은 성종입니다."],
    embedding=OpenAIEmbeddings(),
)

In [ ]:
# 벡터DB 에서 제공하는 유사도 검색 기능
db.similarity_search("TF IDF 에 대하여 알려줘")

In [ ]:
# filter 사용 (해당 곳에서만 찾는다)
db.similarity_search(
    "TF IDF 에 대하여 알려줘", filter={"source":"data/finance-keywords.txt"}
)

In [ ]:
# 문서 추가
from langchain_core.documents import Document

# page_content, metadata, id 지정
db.add_documents(
    [
        Document(
            page_content="안녕하세요! 도큐먼트를 새로 추가해 볼게요",
            metadata={"source":"mydata.txt"},
            id="1"
        )
    ]
)


# ID "1" 인 것 가져오기
db.get("1")

In [ ]:
db.add_texts(
    ["이전에 추가한 Document 를 덮어쓰겠습니다", "덮어쓴 결과가 어떤가요?"],
    metadatas=[{"source":"mydata.txt"}, {"source":"mydata.txt"}],
    ids=["1", "2"],
)

# id="1" 조회 (동일한 아이디로 추가하면 기존 것을 지우고 저장한다)
db.get("1")

In [ ]:
# id="1" 삭제
db.delete(ids=["1"])

In [ ]:
db.get(["1", "2"])

In [ ]:
# where 조건으로 metadata 조회
db.get(where={"source":"mydata.txt"})

In [ ]:
# 컬렉션 초기화
db.reset_collection()

In [ ]:
# 특정값을 넘어가는 유사도만 조회
retriever = db.as_retriever(
    
)